In [288]:
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
import numpy as np
import pandas as pd
import cvxpy as cp
import torch

In [289]:
pen = 0.1
a=0.9
b=0.1
n=5
t= 0.3
t_ref = 0.6*np.ones(n)

In [290]:

Q = np.identity(n)
L = pen * np.identity(n)
I = np.identity(n)
A = np.array([a**(i+1) for i in range(n)])
B = np.array([[a**(i-1) * b for i in range(j, 0, -1)] + [0]*(n-j) for j in range(1, n+1)])

In [291]:
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(cp.quad_form(A*t +B@x - t_ref, I) + cp.quad_form(x, L)), [x <= np.ones(n), -x <= np.zeros(n), (A*t +B@x) <= np.ones(n), -(A*t +B@x) <= np.zeros(n)])

In [292]:
prob.solve()

0.39802316418878303

In [293]:
x.value

array([0.87088543, 0.69774885, 0.54322603, 0.38843083, 0.21444385])

In [294]:
A*t +B@x.value

array([0.35708854, 0.39115457, 0.40636172, 0.40456863, 0.38555615])